<a href="https://colab.research.google.com/github/lazy-wav/Data-Analysis/blob/main/DistilBert_on_gossipcop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install transformers
!pip install --upgrade transformers

In [22]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from transformers import DistilBertTokenizer

from transformers import DistilBertForSequenceClassification

In [23]:
from google.colab import files
uploaded = files.upload()

Saving gossipcop_fake.csv to gossipcop_fake.csv


In [24]:
from google.colab import files
uploaded = files.upload()

Saving gossipcop_real.csv to gossipcop_real.csv


In [25]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import Dataset
import numpy as np

# 1. Load data and label


In [26]:
true_data = pd.read_csv('gossipcop_real.csv')
fake_data = pd.read_csv('gossipcop_fake.csv')
true_data['label'] = 0
fake_data['label'] = 1
data = pd.concat([true_data, fake_data]).sample(frac=1).reset_index(drop=True)

In [27]:
data = pd.concat([true_data, fake_data]).sample(frac=1).reset_index(drop=True)

# Drop rows with missing titles
data = data.dropna(subset=['title'])

# Ensure all titles are strings
data['title'] = data['title'].astype(str)

In [28]:
texts = data['title'].tolist()
labels = data['label'].tolist()


# 2. Train/Val/Test split


In [29]:
train_texts, temp_texts, train_labels, temp_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42)

# 3. Tokenizer


In [30]:
def tokenize_fn(texts):
    if not isinstance(texts, list):
        texts = texts.tolist()  # In case input is a Series or ndarray
    return tokenizer(texts, truncation=True, padding=True, max_length=64)

# Apply tokenization
train_encodings = tokenize_fn(train_texts)
val_encodings = tokenize_fn(val_texts)
test_encodings = tokenize_fn(test_texts)

# 4. Convert to Hugging Face Dataset format

In [31]:
train_dataset = Dataset.from_dict({**train_encodings, 'labels': train_labels})
val_dataset = Dataset.from_dict({**val_encodings, 'labels': val_labels})
test_dataset = Dataset.from_dict({**test_encodings, 'labels': test_labels})

# 5. Load Model

In [32]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 6. Define metric function


In [33]:

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = (predictions == labels).mean()
    return {'accuracy': accuracy}

# 7. Training arguments


In [34]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir='./logs',
)


In [35]:

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate(test_dataset)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: s-sakshi9317 (s-sakshi9317-na) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.412500
1000,0.359500


{'eval_loss': 0.3409386873245239,
 'eval_accuracy': 0.8681120144534779,
 'eval_runtime': 200.3025,
 'eval_samples_per_second': 11.053,
 'eval_steps_per_second': 0.349,
 'epoch': 1.0}

In [36]:
from transformers import DistilBertTokenizerFast
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=64)
    inputs = {key: val.to(model.device) for key, val in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_class = torch.argmax(outputs.logits, dim=1).item()
    return "FAKE" if predicted_class == 1 else "REAL"

# Example:
print(predict("Breaking: Scientists discover new planet in our solar system"))


REAL


In [37]:
predictions_output = trainer.predict(test_dataset)
logits = predictions_output.predictions
true_labels = predictions_output.label_ids
pred_labels = np.argmax(logits, axis=1)


In [38]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

print("Accuracy:", accuracy_score(true_labels, pred_labels))
print("\nClassification Report:")
print(classification_report(true_labels, pred_labels, target_names=["Real", "Fake"]))
print("\nConfusion Matrix:")
print(confusion_matrix(true_labels, pred_labels))


Accuracy: 0.8681120144534779

Classification Report:
              precision    recall  f1-score   support

        Real       0.88      0.96      0.92      1671
        Fake       0.82      0.60      0.69       543

    accuracy                           0.87      2214
   macro avg       0.85      0.78      0.80      2214
weighted avg       0.86      0.87      0.86      2214


Confusion Matrix:
[[1598   73]
 [ 219  324]]


In [39]:
import shutil
import os
import zipfile

# Save the model locally
save_path = './saved_model'
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

# Zip the folder
zip_path = shutil.make_archive('saved_model', 'zip', save_path)

# Download the zipped model
from google.colab import files
files.download(zip_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [40]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [41]:
save_path = '/content/drive/MyDrive/saved_model'
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"Model saved to: {save_path}")


Model saved to: /content/drive/MyDrive/saved_model


In [42]:
import os

print(os.listdir('./saved_model'))


['tokenizer.json', 'config.json', 'vocab.txt', 'special_tokens_map.json', 'model.safetensors', 'tokenizer_config.json']
